# Pandas : des bonnes pratiques

**Francis Wolinski**

Consultant scientifique indépendant depuis 2013, bénéficiaire du programme résidentiel de **datacraft**
- Audit, Conseil et Projets en Data Science
- Formations professionnelles et enseignement de Python au niveau Master

Février 2022

## Plan

1. **Introduction**

2. **Programmation avec pandas**

3. **Chargement des données**

4. **Autres datasets**

5. **Conclusion**

## 1. Introduction

L'objectif de cet atelier n'est pas d'effectuer une introduction à **pandas** mais plutôt de présenter quelques caractéristiques de la librairie, ainsi que les enseignements tirés d'une utilisation intensive et dans de nombreux contextes depuis 2015.

Cet atelier est librement inspiré par mes lectures sur **pandas**, en particulier, le livre *Effective Pandas* de Matt Harrison, sorti en décembre 2021.

Librairies et versions utilisées dans ce notebook :
- IPython 8.0.1
- numpy 1.22.1
- pandas 1.4.0
- matplotlib 3.5.1
- sparklines 0.4.2

In [ ]:
# load image from pandas books
from IPython.display import Image

Image("images/pandas_books.jpg", width=480, height=266)

**Données de Stack Overflow**

**pandas** est sans doute la librairie Python la plus mentionnée dans *Stack Overflow* sur la période 2018-2021.

In [ ]:
# load image from Stack Overflow
from IPython.display import SVG

SVG("images/stackoverflow.svg")

Sources :
- *pandas: powerful Python data analysis toolkit*, github, 2022, https://github.com/pandas-dev/pandas
- Kevin Markham, *What's the future of the pandas library?*, 2018, https://www.dataschool.io/future-of-pandas/

**Méthodologie CRISP-DM**

**CR**oss-**I**ndustry **S**tandard **P**rocess for **D**ata **M**ining

Etablie par la société SPSS (Statistical Package for the Social Sciences) en 2000 qui a été rachetée par IBM en 2009.

In [ ]:
# load CRISP-DM
Image("images/schema-crisp-dm.png", width=960, height=532)

La librairie **pandas** peut être notamment mise en oeuvre dans les processus : *Data Understanding*, *Data Preparation* et aussi *Modeling* (directement et indirectement via **matplotlib**, **scikit-learn** et autres).

In [ ]:
# load CRISP-DM
Image("images/process-crisp-dm.png", width=960, height=532)

Source :
- *CRISP-DM 1.0 - Step-by-step data mining guide*, 2000, https://www.the-modeling-agency.com/crisp-dm.pdf

**ADN de la librarie**

Pour mémoire, le nom **PANDAS** vient de l'acronyme **PAN**el (3D) + **DA**taframe (2D) + **S**eries (1D). La classe *Panel* a disparu (dépréciée en 2017 et supprimée en 2019), mais le nom de la librairie a été conservé !

Il s'avère que la librairie évolue en permanence (ajouts, modifications, dépréciations) :

version | date
-|-
 0.25 | octobre 2019
1.0.0 | janvier 2020
1.1.0 | juillet 2020
1.2.0 | décembre 2020
1.3.0 | juillet 2021
1.4.0 | juillet 2022

Les conséquences sont :
- Obsolescence rapide des connaissances du data scientist
- Nécessité de gérer un environnement virtuel par projet


## 2. Programmation avec pandas

Avec **pandas**, on est souvent amené à effectuer des séquences d'opérations sur des objets de type *Series* ou *DataFrame*.

Il existe plusieurs styles de programmation. Le style en vogue depuis quelques années est *Method chaining*.

### 2.1 Inplace parameter

Dans ce style de programmation, on modifie l'objet sur place à chaque étape.

```python
df.method1(inplace=True)
df.method2(inplace=True)
df.method3(inplace=True)
```

> <span style="background-color:yellow">The pandas core team discourages the use of the inplace parameter.</span>

En fait, l'utilisation de l'option `inplace=True` ne garantit pas l'absence de copie en mémoire. C'est dû à la gestion de la mémoire par le *BlockManager* qui éclate les données d'un *DataFrame* en autant de *ndarrays* par type de données (int, float, object...).

Source :
- Uwe Korn, *The one pandas internal I teach all my new colleagues: the BlockManager*, 2020 https://uwekorn.com/2020/05/24/the-one-pandas-internal.html

### 2.2 Variable assignment

Dans ce style de programmation, on affecte à une variable le résultat de chaque opération. Ce style peut conduire à la création de nombreuses variables intermédiaires qui sont peu utilisées.

```python
df = df.method1()
df = df.method2()
df = df.method3()
```

### 2.3 Method chaining

Dans ce style de programmation, on enchaîne systématiquement les opérations au fur et à mesure sur l'objet qui résulte de l'opération précédente. L'ensemble des enchaînements est encapsulé entre parenthèses pour des raisons syntaxiques.

```python
(df.method1()
 .method2()
 .method3()
)
```

<span style="background-color:yellow">The pandas core team now encourages the use of "method chaining"</span>. This is a style of programming in which you chain together multiple method calls into a single statement. This allows you to pass intermediate results from one method to the next rather than storing the intermediate results using variables.

Sources:

- Matt Harrison, *Effective Pandas*, 2021, https://store.metasnake.com/effective-pandas-book

- Bindi Chen, *Using Pandas Method Chaining to improve code readability - A tutorial for the best practice with Pandas Method Chaining*,  2020 https://towardsdatascience.com/using-pandas-method-chaining-to-improve-code-readability-d8517c5626ac

- Adiamaan Keerthi, *The Unreasonable Effectiveness of Method Chaining in Pandas*, 2019, https://towardsdatascience.com/the-unreasonable-effectiveness-of-method-chaining-in-pandas-15c2109e3c69

- Kevin Markham, *What's the future of the pandas library?*, 2018, https://www.dataschool.io/future-of-pandas/

- Marc Garcia, *Towards Pandas 1.0*, PyData London Meetup #47, August 2018 https://www.youtube.com/watch?v=hK6o_TDXXN8

Il existe des méthodes spéciales qui facilitent ou permettent le chaînage des instructions avec **pandas** en utilisant essentiellement une **notation fonctionnelle** :

- **En particulier**
    - `s.loc[lambda s_: ...]` ou `df.loc[lambda df_: ...]` : sélections fonctionnelles sur les objets de type *Series* ou *DataFrame* sans avoir à désigner explicitement les objets sur lesquels portent les sélections,
    - `df.assign(col=lambda df_: ...)`  ou `assign({col=lambda df_: ...})`: utilisation de mot-clés, ou d'un dictionnaire, généralisant l'usage d'une notation fonctionnelle pour modifier ou ajouter des colonnes à un *DataFrame*,
    - `s.pipe(func, *args, **kwargs)` ou `df.pipe(func, *args, **kwargs)`: application d'une fonction prenant en premier argument un objet de type *Series* ou *DataFrame*,


- **Egalement**
    - `eq()`, `ne()`, `gt()`, `ge()`, `lt()`, `le()`... : pour toutes les comparaisons binaires sur les objets de type *Series* ou *DataFrame*,
    - `add()`, `sub()`, `mul()`, `div()`, `mod()`, `pow()`... : pour toutes les opérations binaires sur les objets de type *Series* ou *DataFrame*,
    - `where(cond, other)` (resp. `mask(cond, other)`) : remplace les valeurs d'un objet de type *Series* ou *DataFrame* si la condition est fausse (resp. vraie).
    - `transform(func, *args, **kwargs)` : modification d'un objet de type *Series* ou *DataFrame* par application d'une fonction s'appliquant à chacune de ses valeurs,
    - etc.

**Remarque** : Pour les *DataFrames*, il est toujours possible d'avoir des noms de colonnes quelconques, mais la notation pousse un peu à utiliser des noms de colonnes utilisables comme attributs. Exemple : `df.col` au lieu de `df["col"]`.

### 2.4 Un premier exemple

On utilise les données de l'Insee sur les prénoms : https://www.insee.fr/fr/statistiques/2540004?sommaire=4767262

> Le fichier des prénoms contient des données sur les prénoms attribués aux enfants nés en France entre 1900 et 2020.

In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# display options
pd.set_option("display.min_rows", 16)
pd.set_option("display.max_columns", 36)

On charge les données et on les prépare en *variable assignment* (qu'on passera en *method chaining* un peu plus loin). On obtient un *DataFrame* avec le nombre de naissances en France par année depuis 1900, par genre et par prénom.

In [ ]:
# load and process nat2020_csv.zip
df_names = pd.read_csv('data/nat2020_csv.zip',
                  sep=';',
                  header=0,
                  names=['gender', 'name', 'year', 'births'],
                  na_values={"name":"_PRENOMS_RARES", "year":"XXXX"},
                  keep_default_na=False)

# prep names with variable assignment
def prep_names0(df):
    df = df.copy()
    df = df.dropna()
    df = df.loc[df["name"].str.len() > 1]
    df["gender"] = df["gender"].map({1:"M", 2:"F"})
    df["name"] = df["name"].str.title()
    df = df.astype({'gender':'category', 'year':'uint16', 'births':'uint16'})
    df = df[["year", "name", "gender", "births"]]
    df = df.sort_values(["year", "gender", "births", "name"], ascending=[True, True, False, True])
    df = df.reset_index(drop=True)
    return df

df_names0 = prep_names0(df_names)
df_names0

**Exemple résolu**
- Graphique avec l'évolution dans le temps de la diversité des prénoms qui se terminent par une lettre donnée, et ce, pour les 7 lettres qui présentent le plus de diversité la dernière année.

**En variable assignment**

En *variable assignment*, on utilise des variables intermédiaires qui sont utilisées peu de fois et qui encombrent la mémoire de Python (*private heap*), et aussi celle du data scientist !

In [ ]:
# variable assignment
df_names1 = df_names0.copy()
df_names1['terminal'] = df_names1['name'].str[-1].str.upper()  # plus efficace apply(lambda x: x[-1].upper())
tab = df_names1.pivot_table(values='name',
                            index='year',
                            columns='terminal',
                            aggfunc='count',
                            fill_value=0)
cols = tab.iloc[-1].nlargest(7).index
ax = tab[cols].plot.line(title='Diversité des prénoms par année et par lettre terminale')
ax.legend(bbox_to_anchor=(1.05, 1.0));

**En method chaining**

Aucune variable intermédiaire n'est utilisée.

In [ ]:
# method chaining
(df_names0.assign(terminal=lambda df_: df_.name.str[-1].str.upper())  # plus efficace apply(lambda x: x[-1].upper())
 .pivot_table(values='name',
              index='year',
              columns='terminal',
              aggfunc='count',
              fill_value=0)
 .pipe(lambda df_: df_[df_.iloc[-1].nlargest(7).index])
 # plot
 .plot.line(title='Diversité des prénoms par année et par lettre terminale')
 .legend(bbox_to_anchor=(1.05, 1.0))
);

**Remarque** : En *method chaining*, on peut facilement afficher et comprendre les calculs intermédiaires en commentant et en décommentant les instructions (et en suprimant le caractère ";" à la fin).

**Exemple**

- Implémenter une fonction en *method chaining* qui produit un graphique avec l'évolution du nombre de naissances d'un prénom et d'un genre au fil des ans.

In [ ]:
# %load exemples/01_plot_name_gender.py
# Graphique avec le nombre de naissances d'un prénom et d'un genre
def plot_name_gender(df, name, gender):
    pass


In [ ]:
# plot_name_gender
plot_name_gender(df_names1, "Alice", "F");

**Exemple**
- Passer la fonction `prep_names0` en *method chaining*

```python
# prep names with variable assignment
def prep_names0(df):
    df = df.copy()
    df = df.dropna()
    df = df.loc[df["name"].str.len() > 1]
    df["gender"] = df["gender"].map({1:"M", 2:"F"})
    df["name"] = df["name"].str.title()
    df = df.astype({'gender':'category', 'year':'uint16', 'births':'uint16'})
    df = df[["year", "name", "gender", "births"]]
    df = df.sort_values(["year", "gender", "births", "name"], ascending=[True, True, False, True])
    df = df.reset_index(drop=True)
    return df
```

In [ ]:
# %load exemples/02_prep_names
# method chaining
def prep_names(df):
    pass


In [ ]:
# prep_names
df_names1 = prep_names(df_names)
df_names1

**Exemple**
- On a implémenté une fonction qui affiche l'évolution au fil des ans du genre d'un prénom
- Passer la fonction `plot_gender_evolution0` en *method chaining*

```python
# évolution du genre d'un prénom
def plot_gender_evolution0(df, name):
    selection = df.loc[df['name']==name]
    ratio = df.pivot_table(values="births",
                           index="year",
                           columns="gender",
                           aggfunc="sum",
                           fill_value=0)
    evolution = ratio.div(ratio.sum(axis=1), axis=0)
    evolution.plot.line(title=f'Evolution du genre de {name} au fil des ans')
```

- La tester avec différents prénoms :  *Alix*, *Camille*, *Dominique*, *Charlie*, *Noa*, *Claude*, *Kim*, *Jo*, *George*, ...

In [ ]:
# %load exemples/03_plot_gender_evolution
# gender evolution graph for a name
def plot_gender_evolution(df, name):
    pass

In [ ]:
plot_gender_evolution(df_names1, "Camille");

**Autres exemples (merci à Julien &#9786;)**

- Implémenter une fonction qui sélectionne les N prénoms les plus donnés pendant une décennie.
- Implémenter une fonction qui produit un graphique avec l'évolution au fil des ans des naissances des N prénoms les plus donnés pendant une décennie. La tester avec différentes périodes.

In [ ]:
# %load exemples/04_top_names_decade.py
# Top N prénoms de la décennie
def topn_names_decade(df, year, n=5):
    pass


In [ ]:
topn_names_decade(df_names1, 1900)

In [ ]:
# %load exemples/05_plot_topn_names_decade.py
# Top N prénoms de la décennie
def plot_topn_names_decade(df, year, n=5):
    pass

In [ ]:
# decade by decade
for year in range(1900, 2020, 10):
    plot_topn_names_decade(df_names1, year)

## 3. Chargement des données

Il existe de nombreuses fonctions de chargement des données selon le format d'origine, y compris des formats de logiciels propriétaires ; ce qui a certainement contribué au succès de la librairie.

Source : https://pandas.pydata.org/docs/user_guide/io.html

Dans le cadre de cet atelier, seule la fonction `read_csv()` sera abordée.

### 3.1 Interprétation automatique des valeurs manquantes

Par défaut, **pandas** interprète automatiquement certaines chaînes de catactères comme étant des valeurs manquantes. 

In [ ]:
# default NaN values
print(pd._libs.parsers.STR_NA_VALUES)

**Remarque** : &#9888; L'interprétation automatique de certaines valeurs peut gêner voir fausser la phase de *Data Understanding*. On va en voir quelques exemples.

**Exemple**

On charge le dataset des prénoms français avec les valeurs manquantes par défaut.

In [ ]:
# load with default missing values
df_names1 = pd.read_csv('data/nat2020_csv.zip',
                         sep=';')
na1 = df_names1.isna().sum()
na1

On charge le même dataset avec le type `str` et en neutralisant les valeurs manquantes par défaut (chaîne vide uniquement).

In [ ]:
# load without missing values except ""
df_names0 = pd.read_csv('data/nat2020_csv.zip',
                  sep=';',
                  dtype=str,
                  na_values="",
                  keep_default_na=False)
na0 = df_names0.isna().sum()
na0

Calcul des écarts entre les nombres de valeurs manquantes.

In [ ]:
# differences of missing values
na1 - na0

Quels noms sont considérés comme des valeurs manquantes ?

In [ ]:
# which names are interpretted as NaN
((df_names0["preusuel"].value_counts() - df_names1["preusuel"].value_counts())
 .loc[lambda s_: s_ != 0]
 .index
 .tolist()
)

Dans ce dataset, *NA* est un prénom d'origine chinoise.

**Exemples**

- On considère le dataset "cities500.zip" : villes mondiales fournies par le site https://www.geonames.org/
- Charger ce dataset sans interprétation des valeurs manquantes (à l'exception de la chaîne vide "").
- Calculer les écarts des nombres de valeurs manquantes entre les 2 *DataFrames*.
- Quels sont les "country_code" et les "admin2_code" qui sont interprétés comme des valeurs manquantes.
- Pour les "admin2_code" interprétés comme des valeurs manquantes, quels sont les "country_code" des pays concernés ?

In [ ]:
# load with default missing values
df_cities1 = pd.read_csv('data/cities500.zip',
                         sep='\t',
                         header=None,
                         dtype=str,
                         names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature_class', 'feature_code', 'country_code', 'cc2', 'admin1_code', 'admin2_code', 'admin3_code', 'admin4_code', 'population', 'elevation', 'dem', 'timezone', 'modification_date'])
df_cities1.shape

In [ ]:
# %load exemples/06_missing_values.py
#

**Conclusion** : En phase de découverte d'un dataset, pour éviter des interprétations erronées, il vaut mieux utiliser de prime abord les trois options :
```python
dtype=str,
na_values="",
keep_default_na=False,
```

**Remarque** : Il est possible d'utiliser `functools.partial` pour définir une fonction de chargement des données brutes à partir de la fonction `pandas.read_csv` en fixant certains arguments.

In [ ]:
# use functools.partial
from functools import partial

load_raw_csv = partial(pd.read_csv, dtype=str, na_values="", keep_default_na=False)

In [ ]:
# usage of load_raw_csv
(load_raw_csv("data/nat2020_csv.zip", sep=";")
 .isna()
 .sum()
)

### 3.2 Visualisation des valeurs manquantes en utilisant la stylisation des *DataFrames*

Un *DataFrame* est doté d'un objet *Styler* accessible par l'opérateur `style` qui permet de styliser l'affichage dans un notebook à l'aide de différentes méthodes :
- `format({"col":"{:}")` : formatte les valeurs des cellules,
- `bar(color="", subset=cols)` : produit un graphique à barres en fonction de la valeur des cellules numériques indiquées (par ex. "lightgreen")
- `background_gradient(cmap="", subset=cols)` : utilise une colormap en fonction de la valeur des cellules numériques indiquées (par ex. "RdYlGn")
- `applymap(func, subset=cols)` : applique une fonction de mise en forme aux cellules en fonction de leur valeur
- `pipe(func, *args, **kwargs)` : applique une fonction à l'objet *Styler*.
- etc.

Couleurs disponibles :
- List of named colors: https://matplotlib.org/stable/gallery/color/named_colors.html
- Choosing Colormaps in Matplotlib: https://matplotlib.org/stable/tutorials/colors/colormaps.html

**Exemples** :
- On charge le fichier "List0F.zip". Il s'agit d'un fichier avec les organismes de formation. On va juste s'intéresser au taux de remplissage des différentes colonnes.
- A partir du nombre de valeurs non nulles des colonnes, styliser un *DataFrame* avec une barre de couleur.
- A partir du pourcentage de remplissage des colonnes arrondi à un chiffre, styliser un *DataFrame* avec une colormap.

In [ ]:
# load ListeOF
df_of = load_raw_csv("data/ListeOF.zip")
df_of.head()

In [ ]:
# %load exemples/07_style_bar.py
#

In [ ]:
# %load exemples/08_style_cmap.py
# 

### 3.3 Optimisation des types numériques et catégoriels

La librairie **pandas** n'optimise pas les types des colonnes chargées. Les types par défaut sont : *int64*, *float64* et *object* pour les chaînes de caractères.

**Chargement brut**

In [ ]:
# load cities500
df_cities0 = pd.read_csv('data/cities500.zip',
                   sep='\t',
                   header=None,
                   names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature_class', 'feature_code', 'country_code', 'cc2', 'admin1_code', 'admin2_code', 'admin3_code', 'admin4_code', 'population', 'elevation', 'dem', 'timezone', 'modification_date'],
                   dtype={'admin1_code': str, 'admin2_code': str, 'admin3_code': str, 'admin4_code': str},
                   na_values=['', -9999],
                   keep_default_na=False)

df_cities0.info(memory_usage="deep")

In [ ]:
# memory_usage
df_cities0.memory_usage(deep=True)

**Optimisation des différents types**

On implémente une fonction qui fournit un type minimaliste pour chaque colonne.

Evidemment encore faut-il que les valeurs des colonnes ne soient pas ensuite amenées à évoluer et à dépasser les limites des différents types.

In [ ]:
# exemple de dépassement de limite
print([i**3 for i in range(10)])

# uint8 : 0-255
(pd.Series(range(10), dtype='uint8')
.pow(3)
)

In [ ]:
# optimized types
def optimized_types(df, n_cats=255):
    """Return a dict with optimized dtypes for a given DataFrame"""
    
    types = {}
    
    # int
    for col in df.select_dtypes(include=['int64']).columns:
        
        s = df[col]
        
        # unsigned int
        if (s >= 0).all():
            for subtype in ['uint8', 'uint16', 'uint32']:  # 'uint64' useless
                if (s <= np.iinfo(subtype).max).all():
                    types[col] = subtype
                    break
                    
        # signed int
        else:
            for subtype in ['int8', 'int16', 'int32']:
                if ((s >= np.iinfo(subtype).min) & (s <= np.iinfo(subtype).max)).all():
                    types[col] = subtype
                    break
                    
    # float
    for col in df.select_dtypes(include=['float64']).columns:
        
        s = df[col].dropna()
        
        for subtype in ['float16', 'float32']:
            if ((s >= np.finfo(subtype).min) & (s <= np.finfo(subtype).max)).all():
                types[col] = subtype
                break
                
    # category
    types.update({col:"category" for col in df.select_dtypes(include=['object']).columns
                   if df[col].nunique() <= n_cats})

    return types
            
optimized_types(df_cities0)

In [ ]:
# memory usage
df_cities1 = df_cities0.astype(optimized_types(df_cities0))
df_cities1.info(memory_usage="deep")

In [ ]:
# memory_usage
df_cities1.memory_usage(deep=True)

**Efficacité de la sélection object vs category**

La sélection de données catégorielles est plus efficace que celle des chaînes de catactères.

In [ ]:
# feature_code list
df_cities0["feature_code"].unique()

In [ ]:
# tests

def tests():
    """Test speed for different feature_code selection"""
    for code in ["PPL", "PPLA", "PPLC"]:  # cities, administrations level 1, capitals
        percent = len(df_cities1.loc[df_cities1["feature_code"]==code])/len(df_cities1)*100
        print(f"{code}: {percent:.1f}%")
        # loc + lambda
        %timeit df_cities0.loc[lambda df_: df_.feature_code==code]
        %timeit df_cities1.loc[lambda df_: df_.feature_code==code]
        print()
        
# tests()

## 4. Autres datasets

### 4.1 Taux de change + sparklines

Dataset des taux de change fourni par la Banque de France : http://webstat.banque-france.fr/fr/

**Chargement du dataset**

In [ ]:
# load Webstat_Export.csv
df_change = pd.read_csv("data/Webstat_Export.csv",
                        sep=";",
                        na_values='-',
                        decimal=',',
                        skiprows=[1, 2])
df_change.shape

**Liste des colonnes**

In [ ]:
# columns
df_change.columns.tolist()

Il est possible d'extraire certains codes ISO3 des devises à l'aide d'une expression régulière.

In [ ]:
# extraction of currency ISO3
(pd.Series(df_change.columns.tolist())
.str.extract(r'\(([A-Z]{3})\)$', expand=False)
.unique()
)

**Préparation du dataset**

On crée une fonction en *variable assignment* pour préparer le dataset et sélectionner quelques devises.

In [ ]:
# prep currency data: variable assignment
def prep_change0(df, currencies):
    df_prep = df.copy()
    cols = pd.Series(df.columns.tolist()).str.extract('\(([A-Z]{3})\)$', expand=False)
    df_prep.columns = ["Date"] + list(cols[1:])
    df_prep["Date"] = pd.to_datetime(df_prep["Date"], format='%d/%m/%Y', errors='ignore')
    df_prep = df_prep.set_index("Date")
    df_prep = df_prep[currencies]
    df_prep = df_prep.dropna()
    df_prep = df_prep.sort_index()
    return df_prep

df_change0 = prep_change0(df_change, ["USD", "CHF", "GBP", "JPY", "RUB", "CNY"])
df_change0

In [ ]:
# Taux de change spots 
fig, ax = plt.subplots(figsize=(10,6))

(df_change0.plot.line(title="Taux de change spots", ax=ax)
 .legend(bbox_to_anchor=(1.05, 1.0))
);

#### Librairie sparklines

Une sparkline est une visualisation de données qui représente la forme générale de l'évolution d'une variable sur une ligne. La sparkline est en général insérée dans un texte et dans un tableau.

Source : 
- https://fr.wikipedia.org/wiki/Sparkline

In [ ]:
# import
import sparklines

# example
print(sparklines.sparklines(pd.Series(range(8)))[0])

On affiche une sparkline donnant l'évolution trimestrielle du dollar en 2019.

In [ ]:
# évolution trimestrielle du dollar en 2019
tab = (df_change0.loc["2019", "USD"]
                 .resample('Q')
                 .mean()
      )
tab

In [ ]:
# sparkline donnant l'évolution trimestrielle du dollar en 2019
print(sparklines.sparklines(tab)[0])

Il est possible de simplifier l'API de sparklines.

In [ ]:
# sparkl
def sparkl(series):
    """Return a sparkline string for the given Series object"""
    return sparklines.sparklines(series)[0]

In [ ]:
sparkl(tab)

**Exemples**
- Passer la fonction `prep_change0` en *method chaining*.
- Modifier le graphique en divisant les taux de change a) par leurs moyennes respectives b) par leurs dernières valeurs respectives.
- Vérifier que le graphique "fonctionne" lorsqu'on restreint la période temporelle considérée (par ex. `loc["2019":"2020"]`).
- Faire un graphique avec les taux de change divisés par leur moyennes respectives a) avec une moyenne mobile de 30 jours b) avec un maximum mobile de 100 jours.
- Produire un *DataFrame* avec les moyennes annuelles du cours du dollar (par exemple) arrondies à 3 décimales et des sparklines donnant les tendances trimestrielles.
- Rajouter un style pour que les moyennes annuelles apparaissent en vert si elles sont supérieures à 1.0 et en rouge sinon.
- Ecrire une fonction qui pour une année donnée produit un *DataFrame* avec les moyennes annuelles des cours des différentes devises et des sparklines donnant leurs tendances trimestrielles.

In [ ]:
# %load exemples/09_prep_change
def prep_change(df, currencies):

    pass

df_change1 = prep_change(df_change, ["USD", "CHF", "GBP", "JPY", "RUB", "CNY"])


In [ ]:
# %load exemples/10_plot_change.py
# 

### 4.2 Online Retail

Dataset *Online Retail* de Kaggle : https://www.kaggle.com/vijayuv/onlineretail

- Ela Kapoor, *Time series and feature engg analysis for retail*, (2021) https://www.kaggle.com/elakapoor/time-series-and-feature-engg-analysis-for-retail

**Exemples**
- Charger le dataset avec la fonction `load_raw_csv` et vérifier les valeurs manquantes.
- Ecrire une fonction de préparation du dataset :
    - suppression des lignes dupliquées
    - utiliser des types optimisés pour "Quantity" et "UnitPrice" (float32)
    - convertir "InvoiceDate" en date
- Faire un graphique à barres avec les volumes des transactions selon les heures
- Faire des graphiques à secteurs avec les volumes et les montants des transactions "UK" / "Non UK"
- Faire des graphiques à barres avec les volumes et les montants des transactions par pays "Non UK"
- Faire des graphiques à barres avec les volumes et les montants des transactions par type "Purchase" (Quantity &geq; 0) ou "Return" (Quantity &lt; 0)
- Produire un *DataFrame* avec les moyennes des montants des transactions par pays arrondies à 2 décimales triées par ordre décroissant et des sparklines donnant les tendances trimestrielles.

In [ ]:
# load raw CSV
df_or = load_raw_csv("data/OnlineRetail.csv.zip", encoding='unicode_escape')
df_or.head()

In [ ]:
# %load exemples/11_prep_retail

def prep_retail(df):
    pass

df_or = prep_retail(df_or)
df_or.shape

In [ ]:
# %load exemples/12_plot_retail.py


## 5. Conclusion

Voir si la *Method chaining* tient ses promesses :
- Normaliser l'écriture du code avec *pandas*
- Faciliter la compréhension et la maintenance du code
- Optimiser l'utilisation de la mémoire

**Autres ateliers potentiels**

- Python et Excel : librairies pandas, xlwings, openpyxl
- Dashboarding en Python avec plotly-dash
- Autres sujets...

Animateurs ou co-animateurs bienvenus !